In [11]:
import requests
import json
import time

with open("key.txt", 'r') as f:
    key = f.read()
print(key)

3CA293064B9ECCDC101642C75CCD74DE


In [13]:
#This will only get us recent 500 matches
def retrieve_recent_match_data(key, start_at=None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key)
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" 
                         + str(start_at) + "&key=" + key)
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)

In [19]:
private_id = 4294967295 #players that we can not get info from
unique_id = set([])
least_recent = None
leave_loop = False
while len(unique_id) < 30000 and leave_loop is False:
    matches = retrieve_recent_match_data(key, start_at = least_recent)
    if matches['result']['results_remaining'] == 0:
        leave_loop = True
    for m in matches['result']['matches']:
        least_recent = m['match_id']
        for p in m['players']:
            if 'account_id' in p and p['account_id'] != private_id:
                unique_id.add(p['account_id'])
    time.sleep(0.9)
    print(len(unique_id))

295
634
983
1337
1630


In [ ]:
def find_player_recent_match(